In [ ]:
import glob
import os
import librosa
import numpy as np
!pip install pretty_midi
import pretty_midi

     |████████████████████████████████| 5.6 MB 5.1 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=821909659bf77167cef2ac5bce6c22ca9206eed848d930bb6f0ce5b182e3281c
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


**Please update the start path and destination path**

In [ ]:
start ='/content/drive/MyDrive/MUS' # Divide all 9 directories of the MAPS Dataset into train/test/val and provide its path (format eg. test/.wav)
dest = '/content/drive/MyDrive/finale' # Destination Path for storing the .npy files

In [ ]:
RangeMIDInotes=[21,108]
n_fft = 502
sr=16000.
bins_per_octave=36
n_octave=7
val_rate=1./7
pretty_midi.pretty_midi.MAX_TICK = 1e10
n_bins= n_octave * bins_per_octave
hop_length = 256
win_width = 32
kernel_size=7
overlap=True

In [ ]:
def midi2mat(midi_path_train, length, spect_len, sr, RangeMIDInotes=RangeMIDInotes):
    midi_data = pretty_midi.PrettyMIDI(midi_path_train)
    pianoRoll = midi_data.instruments[0].get_piano_roll(fs=spect_len * sr/length)
    Ground_truth_mat = (pianoRoll[RangeMIDInotes[0]:RangeMIDInotes[1] + 1, :spect_len] > 0)
    return Ground_truth_mat

In [ ]:
fil = [direc for direc in os.listdir(start)]
for direc in fil:
    j=0
    k=0
    startpath= os.path.join(start,direc)
    destpath = os.path.join(dest,direc)
    if not os.path.exists(destpath):
        os.makedirs(destpath)
    print(direc)
    files = [f for f in os.listdir(startpath)]
    for f in files:
        fpath=startpath
        f1=f
        if 1:
            ffile=os.path.join(fpath,f1)
            file_name,file_extensions=os.path.splitext(f1)
            if file_extensions == '.txt':
                continue
            if file_extensions==".mid":
                ffile=os.path.join(fpath,file_name+'.wav')
            x,sr = librosa.load(ffile,sr=sr)
            
            # perform stft to obtain 2D array of complex values(frequencies)
            stft = librosa.stft(x, n_fft=n_fft, hop_length=hop_length)
            # taking their absolute values
            spect_abs = np.abs(stft)
            # converting the frequency data on a logarthmic scale for better visualisation
            spect = np.transpose(librosa.amplitude_to_db(spect_abs))

            midi_file = os.path.join(fpath,f1)

            if file_extensions==".wav":
                midi_file = os.path.join(fpath,file_name+'.mid')

            Ground_truth_mat=midi2mat(midi_file, len(x), spect.shape[0], sr, RangeMIDInotes=RangeMIDInotes)
            midi_train = np.transpose(Ground_truth_mat)
            
            #midi length<stft length, cut stft
            if midi_train.shape[0]<spect.shape[0]:
                spect=spect[:midi_train.shape[0],:]
        
            if file_extensions == ".wav" :
                ofolder = 'wav'
                subname = 'STFT'
                no=j
            elif file_extensions == ".mid" :
                ofolder = 'mid'
                subname = 'label'
                no=k

            opath = os.path.join(destpath,f,ofolder,file_name)+subname+'.npy'
            temp_path = os.path.join(destpath,f,ofolder)

            if not os.path.exists(temp_path):
                os.makedirs(temp_path)

            if file_extensions == ".wav":
                np.save(opath,spect)
            elif file_extensions == ".mid":
                np.save(opath,midi_train)

            # print('Preprocessed ',f1)   

            # cut the tensor along the first axis by the win_width with a single frame hop
            matrix = np.array(np.load(opath))
            l=matrix.shape[0]
            cut_matrix=[]
            nb_win=int(l/win_width)   #integer division=floor

            if not overlap:
                for i in range(nb_win):
                    cut_matrix.append(matrix[i*win_width:(i+1)*win_width,:])
            else:
                w=matrix.shape[1]
                matrix_1=np.concatenate([np.zeros([int(kernel_size/2),w]),matrix,np.zeros([int(kernel_size/2),w])],axis=0)  #padding
                cut_matrix = []
                for i in range(nb_win):
                    cut_matrix.append(matrix_1[i * win_width:(i + 1) * win_width+kernel_size-1,:])

            cut_matrix = np.asarray(cut_matrix)
            os.remove(opath)
            # print("Removed ",f1)            
            if file_extensions == ".wav":
                if j == 0:
                    X = cut_matrix
                    #print(cut_matrix.shape)
                else:
                    X = np.concatenate((X,cut_matrix),axis=0)
                    #print(cut_matrix.shape)
                j=j+1
                
            elif file_extensions == ".mid":
                if k == 0:
                    Y = cut_matrix
                    #print(cut_matrix.shape)
                else:
                    Y = np.concatenate((Y,cut_matrix),axis=0)
                    #print(cut_matrix.shape)
                k=k+1
            print('Joined ',f1,"no ",no)
            # print('--------------')
            os.rmdir(temp_path)
        os.rmdir(os.path.join(destpath,f))
    X = np.expand_dims(X,axis=-2)
    Y = np.expand_dims(Y,axis=-2) 
    opath1= os.path.join(destpath,"X_final_STFT_")+direc+'.npy'
    opath2= os.path.join(destpath,"Y_final_STFT_")+direc+'.npy' 
    np.save(opath1,X)
    np.save(opath2,Y)
    # print('Saved X_train final')     
    # print('Saved Y_train final')
    # print('X_train_Shape -',X.shape)     
    # print('Y_train_Shape -',Y.shape)

train_source
Joined  MAPS_MUS-alb_se2_StbgTGd2.mid no  0
Joined  MAPS_MUS-bk_xmas1_StbgTGd2.mid no  1
Joined  MAPS_MUS-bk_xmas4_StbgTGd2.mid no  2
Joined  MAPS_MUS-bk_xmas5_StbgTGd2.mid no  3
Joined  MAPS_MUS-bor_ps6_StbgTGd2.mid no  4
Joined  MAPS_MUS-chpn-e01_StbgTGd2.mid no  5
Joined  MAPS_MUS-chpn-p19_StbgTGd2.mid no  6
Joined  MAPS_MUS-deb_clai_StbgTGd2.mid no  7
Joined  MAPS_MUS-deb_menu_StbgTGd2.mid no  8
Joined  MAPS_MUS-grieg_butterfly_StbgTGd2.mid no  9
Joined  MAPS_MUS-liz_et6_StbgTGd2.mid no  10
Joined  MAPS_MUS-liz_et_trans5_StbgTGd2.mid no  11
Joined  MAPS_MUS-liz_rhap09_StbgTGd2.mid no  12
Joined  MAPS_MUS-mz_311_1_StbgTGd2.mid no  13
Joined  MAPS_MUS-mz_331_2_StbgTGd2.mid no  14
Joined  MAPS_MUS-mz_331_3_StbgTGd2.mid no  15
Joined  MAPS_MUS-mz_332_2_StbgTGd2.mid no  16
Joined  MAPS_MUS-mz_333_2_StbgTGd2.mid no  17
Joined  MAPS_MUS-mz_333_3_StbgTGd2.mid no  18
Joined  MAPS_MUS-mz_545_3_StbgTGd2.mid no  19
Joined  MAPS_MUS-mz_570_1_StbgTGd2.mid no  20
Joined  MAPS_MUS-pat